In [1]:
import cv2
import os
import numpy as np

def load_and_preprocess_images_from_directory(directory, target_shape):
    image_list = []
    for root, dirs, files in os.walk(directory):
        for filename in files:
            if filename.endswith(".jpg"):
                img = cv2.imread(os.path.join(root, filename))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
                img = cv2.resize(img, target_shape)  # Resize to the target shape
                image_list.append(img)
    return image_list

# Define the target shape (e.g., 256x256)
target_shape = (256, 256)

# Load grayscale and color images from directories with subfolders
grayscale_images = load_and_preprocess_images_from_directory("bw_output/", target_shape)
color_images = load_and_preprocess_images_from_directory("color_input/", target_shape)
print(len(grayscale_images))

# Convert grayscale images to single-channel format
grayscale_images = [cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) for img in grayscale_images]

# Expand the dimensions of grayscale images to match the model input
grayscale_images = [np.expand_dims(img, axis=-1) for img in grayscale_images]

# Example: Ensure that the color images are in RGB format
for i, img in enumerate(color_images):
    if img.shape[2] == 1:
        # If color images are in grayscale, convert them to RGB
        color_images[i] = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

# Expand the dimensions of color images to match the model input
color_images = [np.expand_dims(img, axis=-1) for img in color_images]

370


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models

def create_colorization_model():
    model = models.Sequential()

    # Encoder
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(256, 256, 1)))
    model.add(layers.MaxPooling2D((2, 2), padding='same'))
    
    # Decoder
    model.add(layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.UpSampling2D((2, 2)))
    
    # Output layer with 3 channels (RGB)
    model.add(layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same'))
    
    return model

In [3]:
import tensorflow as tf

# Create the model
model = create_colorization_model()

# Define the custom loss function
def custom_loss(y_true, y_pred):
    y_true = tf.cast(y_true, dtype=tf.float32)  # Cast y_true to float32
    return tf.reduce_mean(tf.square(y_true - y_pred))

# Compile the model with the updated loss function
model.compile(optimizer='adam', loss=custom_loss)

In [4]:
# Now you can use this compiled model for training and evaluation.

import numpy as np
# Step 4: Training

# Train the model on grayscale and color images
model.fit(np.array(grayscale_images), np.array(color_images), epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
10/10 [==============================] - 49s 5s/step - loss: 20219.8672 - val_loss: 12418.6465
Epoch 2/10
10/10 [==============================] - 49s 5s/step - loss: 20216.0898 - val_loss: 12418.6436
Epoch 3/10
10/10 [==============================] - 48s 5s/step - loss: 20216.0898 - val_loss: 12418.6406
Epoch 4/10
10/10 [==============================] - 51s 5s/step - loss: 20216.0898 - val_loss: 12418.6406
Epoch 5/10
10/10 [==============================] - 51s 5s/step - loss: 20216.0898 - val_loss: 12418.6406
Epoch 6/10
10/10 [==============================] - 50s 5s/step - loss: 20216.0898 - val_loss: 12418.6406
Epoch 7/10
10/10 [==============================] - 50s 5s/step - loss: 20216.0898 - val_loss: 12418.6406
Epoch 8/10
10/10 [==============================] - 50s 5s/step - loss: 20216.0879 - val_loss: 12418.6387
Epoch 9/10
10/10 [==============================] - 51s 5s/step - loss: 20216.0898 - val_loss: 12418.6387
Epoch 10/10
10/10 [===========================

In [7]:
import tensorflow as tf

# Save the trained model
model.save('colorization_model.h5')

# Define your custom loss function
def custom_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

# Load the model and specify the custom loss function
trained_model = tf.keras.models.load_model('colorization_model.h5', custom_objects={'custom_loss': custom_loss})

ValueError: Unknown loss function: 'custom_loss'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.